In [1]:
import numpy as np
# import tensorflow as tf
# tf.get_logger().setLevel('INFO')
import importlib

import random
import os
import sys



In [3]:
import seaborn as sns
from matplotlib.pyplot import scatter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pickle

def rand_jitter(arr):
    stdev = .03*(max(arr)-min(arr))
    return arr + np.random.randn(len(arr)) * stdev

def y_jitter(arr):
    stdev = .08 * (max(arr) - min(arr))
    return arr + np.random.randn(len(arr)) * stdev

def jitter(x, y, s=20, c='b', marker='o', cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, verts=None, hold=None, **kwargs):
    return scatter(rand_jitter(x), y_jitter(y),\
                   s=s, c=c, marker=marker, cmap=cmap, norm=norm, \
                   vmin=vmin, vmax=vmax, alpha=alpha, linewidths=linewidths, \
                   verts=verts)


# np.random.seed(42 * 100 + 4321567)
# base_pca = PCA(n_components=2).fit_transform(base)
# rnd_per = np.array(base_pca)
with open('saved_data_mnist', 'rb') as f:
    saved_result = pickle.load(f)

curr_round = saved_result[0]    
x = curr_round['x']
y = curr_round['y']

# kmeans = KMeans(init='k-means++' , n_clusters=4, n_init=10, max_iter=100)
# kmeans.fit(rnd_per)

# classes = kmeans.predict(rnd_per)
classes = curr_round['class']
print(classes)

jitter(x, y, c=classes)
plt.savefig("cluster_plot.png",  dpi=300, transparent=True)
# plt.scatter(x, y, c=classes, marker="o")
# plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'saved_data_mnist'

In [2]:
def setup_clients(dataset, model=None, use_val_set=False):
    eval_set = 'test' if not use_val_set else 'val'
    train_data_dir = os.path.join('data', dataset, 'data', 'train')
    test_data_dir = os.path.join('data', dataset, 'data', eval_set)

    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
    if len(groups) == 0:
        groups = [[] for _ in users]    
    clients = [Client(u, g, train_data[u], test_data[u], model) for u, g in zip(users, groups)]

    return clients, train_data, test_data




############################## feminst.cnn##############################

In [ ]:
model_path = '%s.%s' % ("femnist", "cnn")
print('############################## %s ##############################' % model_path)
mod = importlib.import_module(model_path)
ClientModel = getattr(mod, 'ClientModel')

path = "../fedmc/models/checkpoints/femnist"
K = 4
lr = 0.01
num_class = 62

model_initlization = ClientModel(123456, lr, num_class)
clients, train_data, test_data = setup_clients("femnist", model_initlization)

############################## celeba.cnn##############################

In [ ]:
model_path = '%s.%s' % ("celeba", "cnn")
print('############################## %s ##############################' % model_path)
mod = importlib.import_module(model_path)
ClientModel = getattr(mod, 'ClientModel')

path = "../fedmc/models/checkpoints/celeba"
K = 3
lr = 0.1
num_class = 2

model_initlization = ClientModel(654321, lr, num_class)
clients, train_data, test_data = setup_clients("celeba", model_initlization)

In [4]:
K = 4
ens_models = []
for i in range(K):
    model = ClientModel(random.randint(1, 10000), lr, num_class)
    model.load_ckp(os.path.join(path, "K-{}".format(K), "K{}-C{}.ckpt".format(K, i+1)))
    ens_models.append(model)

INFO:tensorflow:Restoring parameters from ../fedmc/models/checkpoints/femnist/K-4/K4-C1.ckpt
INFO:tensorflow:Restoring parameters from ../fedmc/models/checkpoints/femnist/K-4/K4-C2.ckpt
INFO:tensorflow:Restoring parameters from ../fedmc/models/checkpoints/femnist/K-4/K4-C3.ckpt
INFO:tensorflow:Restoring parameters from ../fedmc/models/checkpoints/femnist/K-4/K4-C4.ckpt


In [16]:
# with ens_models[0].graph.as_default():
#     print(str(tf.trainable_variables()))

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>, <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'conv2d_1/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'conv2d_2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization_2/gamma:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'batch_normalization_2/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'conv2d_3/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'conv2d_3/bias:0' shape=(32,) dtype=float32_ref>, <

In [10]:
import copy
import pandas as pd

VARIABLE_KEY = "dense_1/kernel"

def make_x_base(m_idx, c_size):
    
    base = list()
    key_dict = {}
    ac_list = [0] * c_size
    id_list = [''] * c_size
    lvl_list = [0] * c_size
#     tmp_model = ClientModel(123456, lr, num_class)
#     tmp_model.load_ckp("./init/rnd_model.ckpt")
    for x, client in enumerate(clients[0:c_size]):
        client.model.set_params(ens_models[m_idx].get_params())
        train(client.model, client)
        accuracy = model_initlization.test(client.eval_data)[ACCURACY_KEY]
        id_list[x] = str(client.id) 
        ac_list[x] = accuracy
        if (accuracy <= 0.5):
            lvl_list[x] = 1
        elif (accuracy <1.0):
            lvl_list[x] = 2
                
        #print("accuracy: % +.2f" % accuracy)
        with client.model.graph.as_default():
            all_vars = tf.trainable_variables()
            for v in all_vars:
                key_dict[v.name] = v.eval(client.model.sess)
            l_w = key_dict[VARIABLE_KEY + ":0"].reshape(1, -1)
            base.extend(l_w)
    print("mean:", np.mean(ac_list))
    print("std:", np.std(ac_list, axis=0))
    df = pd.DataFrame(zip(id_list, ac_list, lvl_list), columns=["id", "acc", "lvl"])
    return base, df

def train(model, client):
        _, params = model.train(train_data[client.id], num_epochs=10, batch_size = 4)

In [11]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

base, df = make_x_base(0, 60)

mean: 0.8595676871163167
std: 0.05714244894871851


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt


# fig = plt.figure(figsize=(8, 8), dpi=200)
# N = 100
# r0 = 0.6
# x = 0.9 * np.random.rand(N)
# y = 0.9 * np.random.rand(N)
# area = (20 * np.random.rand(N))**2  # 0 to 10 point radii
# c = np.sqrt(area)
# r = np.sqrt(x ** 2 + y ** 2)
# area1 = np.ma.masked_where(r < r0, area)
# area2 = np.ma.masked_where(r >= r0, area)
# plt.scatter(x, y, s=area1, marker='^', c=c)
# plt.scatter(x, y, s=area2, marker='o', c=c)